### 01_API key_load_환경변수(environment_variables)사용

In [1]:
import os
from dotenv import load_dotenv

# .env 파일을 불러오기
load_dotenv(r"C:\Users\241011\Documents\key.env")

# 환경 변수 중 하나를 확인 (예: 'KEY'라는 환경 변수 확인)
key = os.getenv("GOOGLE_API_KEY")  # 여기서 "KEY"는 .env 파일에 정의된 환경 변수의 이름입니다.

# 환경 변수가 존재하는지 확인
if key:  # key 값이 존재하면 True, 존재하지 않으면 False
    print("yes key")
else:
    print("no key")


yes key


In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# UnstructuredCSVLoader 생성 (encoding="utf-8" 추가)
loader = CSVLoader(r"C:\Users\241011\Documents\TIL\00_Class\03_RAG\00_data\musinsa_data_6.csv", encoding="utf-8")

# 문서 로드
docs = loader.load()

# 문서 길이 출력
print(len(docs))


2502


In [3]:
for doc in docs:
    print(doc.metadata)  # 시트나 다른 메타데이터 정보

{'source': 'C:\\Users\\241011\\Documents\\TIL\\00_Class\\03_RAG\\00_data\\musinsa_data_6.csv', 'row': 0}
{'source': 'C:\\Users\\241011\\Documents\\TIL\\00_Class\\03_RAG\\00_data\\musinsa_data_6.csv', 'row': 1}
{'source': 'C:\\Users\\241011\\Documents\\TIL\\00_Class\\03_RAG\\00_data\\musinsa_data_6.csv', 'row': 2}
{'source': 'C:\\Users\\241011\\Documents\\TIL\\00_Class\\03_RAG\\00_data\\musinsa_data_6.csv', 'row': 3}
{'source': 'C:\\Users\\241011\\Documents\\TIL\\00_Class\\03_RAG\\00_data\\musinsa_data_6.csv', 'row': 4}
{'source': 'C:\\Users\\241011\\Documents\\TIL\\00_Class\\03_RAG\\00_data\\musinsa_data_6.csv', 'row': 5}
{'source': 'C:\\Users\\241011\\Documents\\TIL\\00_Class\\03_RAG\\00_data\\musinsa_data_6.csv', 'row': 6}
{'source': 'C:\\Users\\241011\\Documents\\TIL\\00_Class\\03_RAG\\00_data\\musinsa_data_6.csv', 'row': 7}
{'source': 'C:\\Users\\241011\\Documents\\TIL\\00_Class\\03_RAG\\00_data\\musinsa_data_6.csv', 'row': 8}
{'source': 'C:\\Users\\241011\\Documents\\TIL\\00_Class

In [4]:
print(docs[0].page_content)

name: 스피드캣 OG - 블랙 / 398846-01
type: shoes
season: spring_summer_autumn_winter
review: 스피드캣 OG - 블랙은 클래식한 디자인과 탁월한 편안함을 자랑하는 운동화입니다. 발을 안정적으로 지지해주며, 스타일리시하면서도 어떤 의상에도 잘 어울립니다. 이 신발은 일상적인 외출뿐만 아니라 간단한 운동에도 적합해 활용도가 높습니다.
price: 139,000원
brand: 푸마
link: https://www.musinsa.com/products/4201442


### 03_TextSplitter

In [5]:
pages = docs

# RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

splits_RCTS = recursive_text_splitter.split_documents(pages)

# Document 객체의 page_content 속성에서 텍스트의 길이를 측정
char_list = []
for i in range(len(splits_RCTS)):
    char_list.append(len(splits_RCTS[i].page_content))
print("chunk_size:")
print(char_list[:20])
print("="*100)

# 상위 50개 청크 선택
top_50_chunks_RCTS = splits_RCTS[:5]  # 순차적으로 상위 50개 청크 선택(제출용 5개)

# 상위 50개 청크 출력
for i, chunk in enumerate(top_50_chunks_RCTS):
    print(f"Chunk {i + 1}: {chunk}")

chunk_size:
[292, 252, 275, 274, 273, 248, 260, 221, 270, 262, 288, 255, 266, 267, 264, 254, 271, 248, 238, 223]
Chunk 1: page_content='name: 스피드캣 OG - 블랙 / 398846-01
type: shoes
season: spring_summer_autumn_winter
review: 스피드캣 OG - 블랙은 클래식한 디자인과 탁월한 편안함을 자랑하는 운동화입니다. 발을 안정적으로 지지해주며, 스타일리시하면서도 어떤 의상에도 잘 어울립니다. 이 신발은 일상적인 외출뿐만 아니라 간단한 운동에도 적합해 활용도가 높습니다.
price: 139,000원
brand: 푸마
link: https://www.musinsa.com/products/4201442' metadata={'source': 'C:\\Users\\241011\\Documents\\TIL\\00_Class\\03_RAG\\00_data\\musinsa_data_6.csv', 'row': 0}
Chunk 2: page_content='name: 810s 마르케 모디 White
type: shoes
season: spring_summer_autumn_winter
review: 810s 마르케 모디 화이트는 깔끔하고 미니멀한 디자인이 특징으로, 다양한 스타일에 잘 어울립니다. 발에 편안하게 맞는 쿠셔닝과 경량 소재로 장시간 착용해도 불편함이 없습니다.
price: 112,000원
brand: 문스타
link: https://www.musinsa.com/products/3813326' metadata={'source': 'C:\\Users\\241011\\Documents\\TIL\\00_Class\\03_RAG\\00_data\\musinsa_data_6.csv', 'row': 1}
Chunk 3: page_content='name: 에어 포스 1 07 M - 블랙 / CW2288-001
type:

### 04_Embeddings

In [6]:
import os
from langchain_huggingface import HuggingFaceEmbeddings

# HuggingFace 모델 캐시 경로 설정 (로컬에 저장)
#os.environ["TRANSFORMERS_CACHE"] = r"C:\Users\241011\Documents\models"
os.environ["HF_HOME"] = r"C:\Users\241011\Documents\models"

# 모델 이름 및 설정
model_name = "jhgan/ko-sbert-nli"  # 한국어 임베딩 모델
#model_kwargs = {'device': 'cpu'}  # CPU에서 모델을 실행
model_kwargs = {'device': 'cuda'}  # GPU에서 모델을 실행
encode_kwargs = {'normalize_embeddings': True}  # 임베딩 정규화

# HuggingFace 임베딩 객체 생성
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# 각 분할된 텍스트에 대해 임베딩 생성
doc_list = [split.page_content for split in splits_RCTS]  # 텍스트만 추출
embeddings_list = embeddings.embed_documents(doc_list)


# 임베딩 결과 출력 (예시)
for idx, embedding in enumerate(embeddings_list[:5]):  # 첫 5개만 출력 예시
    print(f"Embedding {idx + 1}: {embedding[:10]}...")  # 임베딩의 앞부분만 출력

C:\Users\241011\miniconda3\envs\p310\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Embedding 1: [-0.05784798413515091, -0.0004776718560606241, 0.007303408347070217, -0.019194472581148148, 0.0267654899507761, -0.045069482177495956, 0.03397238627076149, -0.009153066202998161, -0.03733539953827858, -0.06689528375864029]...
Embedding 2: [0.0028798221610486507, -0.02626669406890869, 0.008217083290219307, -0.01703747548162937, 0.015295677818357944, -0.03702589124441147, -0.029406415298581123, -0.003317367285490036, -0.026515085250139236, -0.06255441159009933]...
Embedding 3: [-0.03864783048629761, -0.03584680333733559, 0.0155470697209239, -0.010315208695828915, -0.00946618802845478, -0.047152578830718994, -0.022779731079936028, -0.010265248827636242, -0.048240844160318375, -0.07122410088777542]...
Embedding 4: [-0.0070373560301959515, -0.020709270611405373, 0.016406754031777382, -0.015753300860524178, 0.027805309742689133, -0.05207578092813492, -0.044020771980285645, -0.042043451219797134, -0.03144589439034462, -0.05218695104122162]...
Embedding 5: [-0.0797770619392395, -0

### 05_Vectorstores

In [7]:
from langchain_community.vectorstores import FAISS

# FAISS VectorStore 생성
faiss_vectorstore = FAISS.from_documents(documents=splits_RCTS, embedding=embeddings)
# 로컬에 FAISS 인덱스를 저장
faiss_vectorstore.save_local("faiss_index")

In [13]:
# 로컬에 저장된 FAISS 인덱스를 불러오기 (보안 상 신뢰할 수 있으면 True로 설정)
loaded_faiss = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

# 검색기 설정 (가장 유사한 3개의 문서를 검색)
faiss_retriever = loaded_faiss.as_retriever(search_kwargs={"k": 5})

### 06_LLM_model

In [8]:
from IPython.display import Markdown
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-pro")

### 08_RAG+prompt_engineering_load_and_save 

In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_transformers import LongContextReorder 
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

prompt = ChatPromptTemplate.from_messages([("user", """
Please select the most similar information and let me know, even if it is not an exact match.
 Answer: 하이페리엄 슬라이드 블랙은 세련되면서도 실용적인 디자인의 슬리퍼입니다. 블랙 컬러는 어떤 스타일에도 잘 어울려 데일리로 사용하기에 좋으며, 간편하게 착용할 수 있는 슬라이드 형식이 매력적입니다. 가격은 22,000원이고, 브랜드는 블렌도프입니다. 링크는 https://www.musinsa.com/products/1564155 입니다."\n
 Answer: 스피드캣 OG - 블랙은 클래식한 디자인과 탁월한 편안함을 자랑하는 운동화입니다. 발을 안정적으로 지지해주며, 스타일리시하면서도 어떤 의상에도 잘 어울립니다. 이 신발은 일상적인 외출뿐만 아니라 간단한 운동에도 적합해 활용도가 높습니다. 가격은 159,000원이고, 브랜드는 킨입니다. 링크는 https://www.musinsa.com/products/2378989 입니다."\n
 Answer: 어센틱 블랙은 심플하면서도 세련된 디자인을 자랑하는 신발입니다. 편안하고 실용적인 착용감 덕분에 일상적인 외출에 자주 활용할 수 있으며, 다양한 스타일에 잘 어울립니다. 가격은 13,000원이고, 브랜드는 인볼입니다. 링크는 https://www.musinsa.com/products/3489258 입니다."\n
Question: {question}
""")])

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

callback_handler = StreamingStdOutCallbackHandler()

class prompt_debug(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output

def format_docs(docs):
    print("docs:", docs)
    reordering = LongContextReorder()
    reordered_docs = reordering.transform_documents(docs)
    print("reordered_docs:", reordered_docs)
    
    formatted_docs = "\n\n".join(doc.page_content for doc in reordered_docs )
    
    return formatted_docs



rag_chain = (
        {"context": faiss_retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | prompt_debug()
        | model
        | StrOutputParser()
)

result = rag_chain.invoke("블랙 shoes 안정감")

print(result)

import time

def real_time_print(text, delay=0.1):
    """
    주어진 텍스트를 한자 한자 실시간으로 출력하는 함수.
    
    :param text: 출력할 텍스트
    :param delay: 글자 출력 간의 지연 시간 (초 단위)
    """
    for char in text:
        print(char, end='', flush=True)
        time.sleep(delay)  # 각 글자 출력 후 지연 시간을 설정
    print()  # 마지막에는 줄 바꿈을 추가

#real_time_print(result, delay=0.01)



docs: [Document(metadata={'source': 'C:\\Users\\241011\\Documents\\TIL\\00_Class\\03_RAG\\00_data\\musinsa_data_6.csv', 'row': 52}, page_content='name: 하이페리엄 슬라이드 - 블랙 / 100210105\ntype: shoes\nseason: spring_summer_autumn_winter\nreview: 하이페리엄 슬라이드 블랙은 세련되면서도 실용적인 디자인의 슬리퍼입니다. 블랙 컬러는 어떤 스타일에도 잘 어울려 데일리로 사용하기에 좋으며, 간편하게 착용할 수 있는 슬라이드 형식이 매력적입니다. 발을 편안하게 감싸는 디자인과 부드러운 쿠셔닝 덕분에 장시간 착용해도 편안함을 유지하며, 여름철 외출이나 물가에서 활동할 때 유용하게 활용할 수 있습니다. 가벼운 외출이나 여행, 바캉스에 적합한 이 슬라이드는 간편하면서도 스타일리시한 선택이 될 것입니다.\nprice: 59,000원\nbrand: 리복\nlink: https://www.musinsa.com/products/3302916'), Document(metadata={'source': 'C:\\Users\\241011\\Documents\\TIL\\00_Class\\03_RAG\\00_data\\musinsa_data_6.csv', 'row': 45}, page_content='name: 어센틱 - 블랙 / VN000EE3BLK1\ntype: shoes\nseason: spring_summer_autumn_winter\nreview: 어센틱 블랙은 심플하면서도 세련된 디자인을 자랑하는 신발입니다. 편안하고 실용적인 착용감 덕분에 일상적인 외출에 자주 활용할 수 있으며, 다양한 스타일에 잘 어울립니다.\nprice: 69,000원\nbrand: 반스\nlink: https://www.musinsa.com/products/102622'), Document(metadata={'source': 'C